# 추가 맥주 목록 수집

## 실제 구현 함수

In [57]:
import pandas as pd
import numpy as np
import time

import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

pd.set_option('display.max_rows', 100)

In [58]:
url_list = ['https://namu.wiki/w/%EB%A7%A5%EC%A3%BC/%EB%8F%85%EC%9D%BC',
           'https://namu.wiki/w/%EB%A7%A5%EC%A3%BC/%EC%B2%B4%EC%BD%94',
           'https://namu.wiki/w/%EB%A7%A5%EC%A3%BC/%EB%B2%A8%EA%B8%B0%EC%97%90',
           'https://namu.wiki/w/%EB%A7%A5%EC%A3%BC/%EB%84%A4%EB%8D%9C%EB%9E%80%EB%93%9C',
           'https://namu.wiki/w/%EB%A7%A5%EC%A3%BC/%EC%95%84%EC%9D%BC%EB%9E%9C%EB%93%9C',
           'https://namu.wiki/w/%EB%A7%A5%EC%A3%BC/%EC%98%81%EA%B5%AD'
           ]

In [59]:
def list_crawl(url_, k):
    chromedriver = "chromedriver.exe"
    url = url_
    
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    driver.set_window_size(900,900)
    
    time.sleep(2)
    
    beer_names = driver.find_elements_by_css_selector('.wiki-paragraph')
    
    reg = re.compile('\([A-Za-z0-9]+.*\):+')
    
    tmp=[]

    for i in range(len(beer_names)):
        og_name = beer_names[i].text
        name = reg.findall(og_name)
        tmp.append(name)

    ttmp = pd.DataFrame(data=tmp, columns=['맥주이름1'])
    
    ttmp = ttmp[ttmp['맥주이름1'].notnull()]
    
    reg = re.compile('[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|\(|\)|\:][\w\,\.\s\']*')
    reg2 = re.compile('[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|\(|\)|\:]+')
    
    for i in range(len(ttmp)):
        name = ttmp['맥주이름1'].iloc[i]
        ttmp['맥주이름1'].iloc[i] = reg.findall(name)
        
        name = ttmp['맥주이름1'].iloc[i][0]
        ttmp['맥주이름1'].iloc[i] = reg2.findall(name)
    
    for i in range(len(ttmp)):
        ttmp['맥주이름1'].iloc[i] = ttmp['맥주이름1'].iloc[i][0]
    
    driver.quit()
    
    print(k,'번째 저장 완료')
    ttmp.to_csv('extra_beer_list'+str(k)+'.csv', encoding='utf-8')
    
    return ttmp

In [60]:
tmp = list_crawl(url_list[0],0)

0 번째 저장 완료


In [61]:
tmp

,맥주이름1
15,"5,0 Original"
18,Gaffel
20,DAB
22,Diebels Alt
24,Leikeim
26,Löwenbräu
28,Licher
30,Maisel's Weisse Original
32,Warsteiner
38,Berliner Kindl


## 실제 수집

In [62]:
for k in range(len(url_list)):
    list_crawl(url_list[k], k)

0 번째 저장 완료
1 번째 저장 완료
2 번째 저장 완료
3 번째 저장 완료
4 번째 저장 완료
5 번째 저장 완료


# 데이터 병합

In [63]:
data = pd.DataFrame(data=[], columns=['맥주이름1'])

for i in range(len(url_list)):
    tmp = pd.read_csv('extra_beer_list'+str(i)+'.csv', encoding='utf-8', index_col=0)
    data = pd.concat([data,tmp])


In [64]:
data.맥주이름1.unique()

array(['5,0 Original', 'Gaffel', 'DAB', 'Diebels Alt', 'Leikeim',
       'Löwenbräu', 'Licher', "Maisel's Weisse Original", 'Warsteiner',
       'Berliner Kindl', 'Bear Beer', 'Bitburger', 'Schineider',
       'Schmucker', 'Stephans Bräu', 'Spaten', 'Arcobräu',
       'Augustiner Hell', 'einbecker', 'Andechs', 'Erdinger', 'JEVER',
       'Zähringer', 'Cannabis Club', 'Kapuziner', 'Köstritzer',
       'König Pilsener', 'Krombacher', 'Klosterbrauerei Neuzelle',
       'Schwarzer Abt', 'Neuzeller Pilsner', 'Neuzeller Bock', 'Tucher',
       'Paderborner', 'Feldschlößchen', 'Potsdamer Rex Pils',
       'Pfungstädter', 'Fürstenberg', 'Franziskaner', 'Früh', 'Hofbräu',
       'Budweiser Budvar', 'Staropramen', 'Kozel', 'Gambrinus',
       'Krušovice', 'Primator', 'Vratislav', 'Celia', 'Hopfen Brau',
       'ZUBR', 'Litovel', 'RADEGAST', 'Blanche De Bruxelles', 'Mongozo',
       'Willianbräu', 'Eupener', 'Saint Bernardus', 'Abbaye des Rocs',
       'Cantillon', 'Kingdom of Belgium', 'Keller P

In [65]:
data.columns = ['맥주이름']
data.to_csv('extra_beer_list.csv', encoding='utf-8')
data.head()

,맥주이름
15,"5,0 Original"
18,Gaffel
20,DAB
22,Diebels Alt
24,Leikeim


In [66]:
data

,맥주이름
15,"5,0 Original"
18,Gaffel
20,DAB
22,Diebels Alt
24,Leikeim
...,...
33,"MEANTIME LONDON,"
35,"Cobra,"
37,"Boddingtons Pub Ale,"
39,"Belhaven,"


In [67]:
## 데이터 불러오기
data2 = pd.read_csv('추가맥주리스트.csv', encoding='utf-8', index_col=0)
data2.columns = ['맥주이름']

In [68]:
data2.head()

,맥주이름
6,Carlsberg
7,Faxe
8,Fuglsang
9,Mikkeller
10,Tuborg


In [69]:
result = pd.concat([data,data2])

In [70]:
result2 = result.reset_index()

In [71]:
result.head()

,맥주이름
15,"5,0 Original"
18,Gaffel
20,DAB
22,Diebels Alt
24,Leikeim


### 최종 추가 맥주 리스트 저장

In [72]:
result.to_csv('추가맥주리스트.csv', encoding='utf-8')

In [73]:
r_data = pd.read_csv('추가맥주리스트.csv', encoding='utf-8')

In [74]:
r_data.head()

,Unnamed: 0,맥주이름
0,15,"5,0 Original"
1,18,Gaffel
2,20,DAB
3,22,Diebels Alt
4,24,Leikeim


In [58]:
chromedriver = "chromedriver.exe"
url = 'https://namu.wiki/w/%EB%A7%A5%EC%A3%BC/%EC%98%81%EA%B5%AD'

driver = webdriver.Chrome(chromedriver)
driver.get(url)
driver.set_window_size(900,900)

time.sleep(2)

beer_names = driver.find_elements_by_css_selector('.wiki-paragraph')

reg = re.compile('\([A-Za-z0-9]+.*\):+')

tmp=[]

for i in range(len(beer_names)):
    og_name = beer_names[i].text
    name = reg.findall(og_name)
    tmp.append(name)

In [59]:
tmp

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['(Newcastle Brown Ale, 요크셔):'],
 [],
 ["(Fuller's London Pride, 런던 서부):"],
 [],
 ['(Tennents, 스코틀랜드 글래스고):'],
 [],
 ['(Brew Dog, 스코틀랜드 애버딘셔):'],
 [],
 ["(Young's Double Chocolate Stout, 베드포드셔):"],
 [],
 ['(Old Speckled Hen, 서포크):'],
 [],
 ['(HOBGOBLIN, 옥스포드셔):'],
 [],
 ['(Bass, 스트랫포드셔):'],
 [],
 ['(Corvus Stout, 윌트셔):'],
 [],
 ['(MEANTIME LONDON, 런던 그리니치):'],
 [],
 ['(Cobra, 스트랫포드셔):'],
 [],
 ['(Boddingtons Pub Ale, 맨체스터):'],
 [],
 ['(Belhaven, 스코틀랜드 이스트로시언):'],
 [],
 ['(Kestrel, 베드포드):']]

In [60]:
ttmp = pd.DataFrame(data=tmp, columns=['맥주이름1'])

In [61]:
ttmp = ttmp[ttmp['맥주이름1'].notnull()]

reg2 = re.compile('[A-Za-z0-9]+[,.]*[\s]*[A-Za-z0-9]*')

for i in range(len(ttmp)):
    name = ttmp['맥주이름1'].iloc[i]
    ttmp['맥주이름1'].iloc[i] = reg2.findall(name)
#     ttmp['맥주이름1'].iloc[i] = name[1:-2]


for i in range(len(ttmp)):
    ttmp['맥주이름1'].iloc[i] = ttmp['맥주이름1'].iloc[i][0]

driver.quit()

In [62]:
ttmp

,맥주이름1
15,Newcastle Brown
17,Fuller
19,"Tennents,"
21,Brew Dog
23,Young
25,Old Speckled
27,"HOBGOBLIN,"
29,"Bass,"
31,Corvus Stout
33,MEANTIME LONDON


# 추가 맥주 리스트 수집 함수 구현 - 기본 맥주 종류

In [15]:
import pandas as pd
import numpy as np
import time

import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

pd.set_option('display.max_rows', 100)

In [16]:
chromedriver = 'chromedriver.exe'
url = 'https://namu.wiki/w/%EB%A7%A5%EC%A3%BC/%EB%AA%A9%EB%A1%9D#s-2'

In [17]:
driver = webdriver.Chrome(chromedriver)
driver.get(url)
driver.set_window_size(1400, 900)
time.sleep(1)

In [18]:
beer_names = driver.find_elements_by_css_selector('.wiki-list')
beer_names

[<selenium.webdriver.remote.webelement.WebElement (session="76ca60d3704f6e02614adcedf2fef85a", element="921f416d-3a78-4126-9ddd-c82d3e184afe")>,
 <selenium.webdriver.remote.webelement.WebElement (session="76ca60d3704f6e02614adcedf2fef85a", element="ea20e337-0fa4-4e30-805c-471098960dd6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="76ca60d3704f6e02614adcedf2fef85a", element="f2682d60-91d1-4a35-bc2b-2187cc459d71")>,
 <selenium.webdriver.remote.webelement.WebElement (session="76ca60d3704f6e02614adcedf2fef85a", element="2346c5a5-9987-4ead-8cc2-93a3abf8b5ac")>,
 <selenium.webdriver.remote.webelement.WebElement (session="76ca60d3704f6e02614adcedf2fef85a", element="d664d30f-6806-4efe-a6ca-f3212ad1ab9e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="76ca60d3704f6e02614adcedf2fef85a", element="b2b6f2f1-c1c6-4ef7-bae0-a514925def21")>,
 <selenium.webdriver.remote.webelement.WebElement (session="76ca60d3704f6e02614adcedf2fef85a", element="29178844-f2b9-4217-bb9d-ec

In [19]:
bname =beer_names[10].text
bname

'투보그(Tuborg): 덴마크 맥주. 흔히 덴마크 맥주하면 칼스버그나 팍세를 떠오르는데 정작 덴마크 현지에서는 칼스버그나 팍세보다 오히려 투보그가 더 유명하다고 한다. 한국에서도 이마트 등지에서 수입, 판매되고 있다.'

In [20]:
import re

reg = re.compile('\([A-Za-z0-9]+\):')

In [21]:
print(reg.findall(bname))

['(Tuborg):']


In [22]:
tmp=[]

for i in range(len(beer_names)):
    ogname = beer_names[i].text
    name = reg.findall(ogname)
    tmp.append(name)

In [23]:
ttmp = pd.DataFrame(data=tmp, columns=['맥주이름1','맥주이름2'])

In [24]:
ttmp[ttmp['맥주이름2'].notnull()]

,맥주이름1,맥주이름2
85,(Bofferding):,(Mousel):
186,(Desant):,(Slavutich):
246,(Kasztelan):,(Brok):
493,(Bieckert):,(PILSEN):


In [25]:
ttmp.tail()

,맥주이름1,맥주이름2
796,(Duff):,None
797,(PaoPao):,None
798,(GraggyIce):,None
799,None,None
800,None,None


In [26]:
ttmp2 = ['Bofferding','Mousel','Desant','Slavutich',
        'Kasztelan','Brok','Bieckert','PILSEN']

In [27]:
ttmp2 = pd.DataFrame(data=ttmp2, columns=['맥주이름1'])

In [28]:
ttmp = ttmp[ttmp['맥주이름2'].isnull()]

In [118]:
ttmp.drop(columns=['맥주이름2'], inplace=True)

In [119]:
ttmp

,맥주이름1
0,None
1,None
2,None
3,None
4,None
...,...
796,(Duff):
797,(PaoPao):
798,(GraggyIce):
799,None


In [120]:
tmp = pd.concat([ttmp,ttmp2], axis=0)
tmp

,맥주이름1
0,None
1,None
2,None
3,None
4,None
...,...
3,Slavutich
4,Kasztelan
5,Brok
6,Bieckert


In [123]:
import re

reg2 = re.compile('[A-Za-z0-9]+')

In [128]:
tmp = tmp[tmp['맥주이름1'].notnull()]
tmp

,맥주이름1
6,[Carlsberg]
7,[Faxe]
8,[Fuglsang]
9,[Mikkeller]
10,[Tuborg]
...,...
3,[Slavutich]
4,[Kasztelan]
5,[Brok]
6,[Bieckert]


In [132]:
for i in range(len(tmp)):
    beer_name = tmp['맥주이름1'].iloc[i][0]
    tmp['맥주이름1'].iloc[i] = reg2.findall(beer_name)

for i in range(len(tmp)):
    tmp['맥주이름1'].iloc[i] = tmp['맥주이름1'].iloc[i][0]

In [134]:
tmp.to_csv('추가맥주리스트.csv', encoding='utf-8')

# 추가 맥주 리스트 수집 구현 과정2

In [286]:
chromedriver = "chromedriver.exe"
url = 'https://namu.wiki/w/%EB%A7%A5%EC%A3%BC/%EB%8F%85%EC%9D%BC'

driver = webdriver.Chrome(chromedriver)
driver.get(url)
driver.set_window_size(1400, 900)

In [287]:
beer_names = driver.find_elements_by_css_selector('.wiki-list')

In [288]:
reg = re.compile('\([A-Za-z0-9]+\):+')

In [289]:
tmp=[]

for i in range(len(beer_names)):
    og_name = beer_names[i].text
    name = reg.findall(og_name)
    tmp.append(name)

In [290]:
len(tmp)

44

In [291]:
ttmp = pd.DataFrame(data=tmp, columns=['맥주이름1'])
ttmp.head(10)

,맥주이름1
0,None
1,(Gaffel):
2,(DAB):
3,None
4,(Leikeim):
5,None
6,(Licher):
7,None
8,(Warsteiner):
9,None


In [292]:
ttmp = ttmp[ttmp['맥주이름1'].notnull()]
ttmp.head()

,맥주이름1
1,(Gaffel):
2,(DAB):
4,(Leikeim):
6,(Licher):
8,(Warsteiner):


In [298]:
reg2 = re.compile('[A-Za-z0-9]+')

for i in range(len(ttmp)):
    name = ttmp['맥주이름1'].iloc[i]
    ttmp['맥주이름1'].iloc[i] = reg2.findall(name)

In [301]:
ttmp['맥주이름1'].iloc[0]

'Gaffel'

In [294]:
for i in range(len(ttmp)):
    ttmp['맥주이름1'].iloc[i] = ttmp['맥주이름1'].iloc[i][0]

In [295]:
ttmp.head()

,맥주이름1
1,Gaffel
2,DAB
4,Leikeim
6,Licher
8,Warsteiner


# Ratebeer 수집단계

In [208]:
import pandas as pd
import numpy as np
import time

import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [209]:
beer_list = pd.read_excel('convenience_beer.xlsx')
beer_list.head(2)

,인덱스,편의점,검색이름
0,0,CU GS25 7/11 ministop,kloud
1,1,CU GS25 7/11 ministop,fitz super clear


In [217]:
# 추가맥주 리스트
added_beer_list = pd.read_csv('추가맥주리스트.csv', encoding='utf-8')
added_beer_list.columns = ['인덱스', '검색이름']
added_beer_list.head(2)

,인덱스,검색이름
0,0,"5,0 Original"
1,1,Gaffel


In [219]:
added_beer_list

,인덱스,검색이름
0,0,"5,0 Original"
1,1,Gaffel
2,2,DAB
3,3,Diebels Alt
4,4,Leikeim
...,...,...
651,651,Slavutich
652,652,Kasztelan
653,653,Brok
654,654,Bieckert


In [220]:
# 수집할 맥주 종류 기입
added_beers = added_beer_list['검색이름'].to_list()
len(added_beers)

656

In [210]:
# 수집할 맥주 종류 기입
beers = beer_list['검색이름'].to_list()
len(beers)

77

In [221]:
# 데이터 프레임 생성
data = pd.DataFrame(data=[], columns=['맥주정보', '검색이름', '맥주이름'])

In [215]:
chromedriver = 'chromedriver.exe'
url = 'https://www.ratebeer.com/search?tab=beer'

In [216]:
driver = webdriver.Chrome(chromedriver)
driver.get(url)
driver.set_window_size(1400, 900)
time.sleep(1)

## 실제 구현 함수

In [223]:
def crawl(driver, beer, data, k):
    # Check 변수
    check = 0
    # 데이터 프레임 생성
    data = pd.DataFrame(data=[], columns=['맥주정보', '검색이름', '맥주이름'])
    # url
    url = 'https://www.ratebeer.com/search?tab=beer'
    
    # url open
    print('url_open... {0} 맥주 데이터를 수집합니다..'.format(beer))
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    driver.set_window_size(900, 900)
    
    # 맥주 검색
    time.sleep(2)
    element = driver.find_element_by_xpath('//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div/input')
    time.sleep(2)
    element.click()
    time.sleep(2)
    element.send_keys(beer)
    time.sleep(3)
    
    # 상품 선택
    driver.find_element_by_xpath('//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div[2]/a[1]/div/div[2]').click()
    
    # 상품 이름 수집
    time.sleep(3)
    beer_name = driver.find_element_by_css_selector('.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.mt-3.MuiTypography-h4').text
    
    while 1:
        try :
            # 전체 리뷰 개수 수집
            time.sleep(3)
            string = driver.find_element_by_class_name('MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.MuiTypography-h6').text

            # ,가 포함되어 있는지에 대한 로직
            extract = re.compile('[0-9]*,*[0-9]+')
            str_num = extract.findall(string)
            str_num = str_num[0]
            
            print('성공... while문을 탈출합니다.')
            break
        except :
            print('오류 발생.. 다시 시작합니다.')
    
    if ',' in str_num:
        str_num = str_num.split(',')
        str_num = int(str_num[0]+str_num[1])
        num = str_num
    else:
        num = int(str_num)
        
    # Score breakdown 클릭
    time.sleep(3)
    element = driver.find_element_by_xpath('//*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[4]/div/div[2]/div[1]/div[2]')
#     element = driver.find_element_by_xpath('//*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[4]/div/div[2]/div[1]/div[2]')
#     element = driver.find_element_by_css_selector('.MuiButtonBase-root.MuiChip-root.RatingsContainer___StyledChip2-iqbJXI.xGfYx.mr-4.MuiChip-outlined.MuiChip-sizeSmall.MuiChip-clickable')
    time.sleep(3)
    driver.execute_script("arguments[0].click();", element)
    
    page_num = num // 15 + 1


    for i in range(page_num):
        print(i+1, '번째 페이지입니다.')

        # 전체 맥주 정보를 통째로 수집
        time.sleep(3)
        beer_info = driver.find_elements_by_css_selector('.px-4.fj-s.f-wrap')
        
        tmp = []
        
        # 수집한 것을 데이터프레임에 저장
        for i in range(len(beer_info)):
            tmp.append(beer_info[i].text)

        tmp = pd.DataFrame(data=tmp, columns=['맥주정보'])
        tmp['맥주이름'] = beer_name
        tmp['검색이름'] = beer
        data = pd.concat([data, tmp])

        # div, span, title 태그 후 속성은 class 외에도 사용 가능
        try :
            element = driver.find_element_by_xpath('//button[@title="Next page"]/span[@class="MuiIconButton-label"]')
            time.sleep(3)
            driver.execute_script("arguments[0].click();", element)
        except:
            print('마지막 페이지입니다.')

    if num != len(data):
        data = data[:num]
    
    print('리뷰수 : ', num, '수집된 리뷰수 : ', len(data))
    
    result = pd.merge(data, beer_list, on='검색이름', how='left')
    result.to_csv("beer_n_"+str(k)+".csv", encoding='utf-8')
    result.to_excel("beer_n_"+str(k)+".xlsx")
    
    driver.quit()
    
    return result

In [224]:
for k in range(0, 6):
    result = crawl(driver, added_beers[k], data, k)

url_open... 5,0 Original 맥주 데이터를 수집합니다..
성공... while문을 탈출합니다.
1 번째 페이지입니다.
2 번째 페이지입니다.
3 번째 페이지입니다.
4 번째 페이지입니다.
5 번째 페이지입니다.
6 번째 페이지입니다.
7 번째 페이지입니다.
8 번째 페이지입니다.
9 번째 페이지입니다.
10 번째 페이지입니다.
11 번째 페이지입니다.
12 번째 페이지입니다.
리뷰수 :  176 수집된 리뷰수 :  176
url_open... Gaffel 맥주 데이터를 수집합니다..
성공... while문을 탈출합니다.
1 번째 페이지입니다.
2 번째 페이지입니다.
3 번째 페이지입니다.
4 번째 페이지입니다.
5 번째 페이지입니다.
6 번째 페이지입니다.
7 번째 페이지입니다.
8 번째 페이지입니다.
9 번째 페이지입니다.
10 번째 페이지입니다.
11 번째 페이지입니다.


KeyboardInterrupt: 